<a href="https://colab.research.google.com/github/georgeliu1998/sentiment_word_embedding/blob/master/sentiment_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numpy.random import seed
seed(8)

from tensorflow import set_random_seed
set_random_seed(8)

import numpy as np
seed = np.random.RandomState(8)

import urllib.request
import tarfile
import os
import re
from time import time

import numpy as np
import pandas as pd
import spacy

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
!python -m spacy download en_core_web_md


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
download_info = urllib.request.urlretrieve(url)
download_info

('/tmp/tmpv6_m04jf', <http.client.HTTPMessage at 0x7fa3122a3668>)

In [0]:
file_path = download_info[0]

tar = tarfile.open(file_path)
tar.extractall()
tar.close()

In [0]:
os.listdir('/content')

['.config', 'aclImdb', 'sample_data']

In [0]:
os.listdir('/content/aclImdb')

['train', 'imdb.vocab', 'README', 'imdbEr.txt', 'test']

In [0]:
def load_train_test_imdb_data(data_dir):
    """Loads the IMDB train/test datasets from a folder path.
    Input:
    data_dir: path to the "aclImdb" folder.
    
    Returns:
    train/test datasets as pandas dataframes.
    """

    data = {}
    for split in ["train", "test"]:
        data[split] = []
        for sentiment in ["neg", "pos"]:
            score = 1 if sentiment == "pos" else 0

            path = os.path.join(data_dir, split, sentiment)
            file_names = os.listdir(path)
            for f_name in file_names:
                with open(os.path.join(path, f_name), "r") as f:
                    review = f.read()
                    data[split].append([review, score])

    np.random.shuffle(data["train"])        
    data["train"] = pd.DataFrame(data["train"],
                                 columns=['text', 'sentiment'])

    np.random.shuffle(data["test"])
    data["test"] = pd.DataFrame(data["test"],
                                columns=['text', 'sentiment'])

    return data["train"], data["test"]

In [0]:
train_data, test_data = load_train_test_imdb_data('./aclImdb')

train_data.to_csv('./gdrive/My Drive/WorkingDir/sentiment_word_embedding/train.csv')
test_data.to_csv('./gdrive/My Drive/WorkingDir/sentiment_word_embedding/test.csv')

In [0]:
train = pd.read_csv('./gdrive/My Drive/WorkingDir/sentiment_word_embedding/train.csv')
test = pd.read_csv('./gdrive/My Drive/WorkingDir/sentiment_word_embedding/test.csv')

In [0]:
train.head()

,Unnamed: 0,text,sentiment
0,0,"I saw this movie once, and I thought it was OK...",0
1,1,For a film with so much promise it was disappo...,0
2,2,After Garbo's introduction to sound in Clarenc...,1
3,3,"Think of this pilot as ""Hawaii Five-O Lite"". I...",0
4,4,A scientist and his girl friend are out drivin...,0


In [0]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [57]:
# Transform each text into a vector of word counts
vectorizer = CountVectorizer(stop_words="english",
                             preprocessor=clean_text)

training_features = vectorizer.fit_transform(train["text"])    
test_features = vectorizer.transform(test["text"])

# Training
model = LinearSVC()
model.fit(training_features, train["sentiment"])
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(test["sentiment"], y_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

Accuracy on the IMDB dataset: 83.68


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

In [0]:
X_train = np.array(list(train['text'].str.lower().apply(lambda x: nlp(x).vector.tolist())))
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_train.csv", X_train, delimiter=",")

In [0]:
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_train.csv", y_train, delimiter=",")

In [0]:
X_train = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_train.csv", delimiter=",")
y_train = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_train.csv", delimiter=",")
X_test = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_test.csv", delimiter=",")
y_test = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_test.csv", delimiter=",")

In [45]:
# Get the first 1000 samples from train dataset
#X_train, y_train = X_train[:1000,], y_train[:1000]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000, 300), (25000,), (25000, 300), (25000,))

In [49]:
model = Sequential()
model.add(Dense(64, input_dim=300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=100,
          batch_size=128)

score = model.evaluate(X_test, y_test, batch_size=128)
print("Accuracy on the test set is: {}".format(score[1]))

Epoch 1/100
25000/25000 [==============================] - 1s 42us/step - loss: 0.5857 - acc: 0.7167
Epoch 2/100
25000/25000 [==============================] - 1s 29us/step - loss: 0.4721 - acc: 0.7886
Epoch 3/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.4276 - acc: 0.8115
Epoch 4/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.4074 - acc: 0.8229
Epoch 5/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3937 - acc: 0.8285
Epoch 6/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3855 - acc: 0.8342
Epoch 7/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3802 - acc: 0.8358
Epoch 8/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3749 - acc: 0.8395
Epoch 9/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3720 - acc: 0.8401
Epoch 10/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3675 - a

In [0]:
def convert_to_embedding(col):
  """
  Converts a pandas column into a numpy array by concatenating average embedding
  vectors for all rows in the column
  """

  return np.array(list(col.str.lower().apply(lambda x: nlp(x).vector.tolist())))

In [0]:
col = test['text']

t1 = time()
X_test = convert_to_embedding(col)
t2 = time()

t = t2 - t1

t /= 60

print("Finished cross-valiation. Took {:.1f} min.".format(t))

Finished cross-valiation. Took 37.5 min.


In [0]:
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_test.csv", X_test, delimiter=",")

In [0]:
y_test = test['sentiment'].values
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_test.csv", y_test, delimiter=",")

In [41]:
#X_test, y_test = X_test[:1000,], y_test[:1000]
X_test.shape, y_test.shape

((1000, 300), (1000,))

## To-Do's
 
- file download 
- 

In [0]:
def cosine_similarity(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [0]:

from keras.layers import Input,Dense
from keras.models import Model
from keras.datasets import mnist
from keras.utils import np_utils
import numpy as np
 
(data,labels),(x_test,y_test) = mnist.load_data()
 
x_train = data.reshape(len(data),-1)
y_train = np_utils.to_categorical(labels, 10)
 
# This returns a tensor
inputs = Input(shape=(784,))
 
# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
 
# This creates a model that includes the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train)  # starts training

11493376/11490434 [==============================] - 1s 0us/step
Epoch 1/1
60000/60000 [==============================] - 19s 313us/step - loss: 6.1711 - acc: 0.6105


In [0]:
data.shape, labels.shape

((60000, 28, 28), (60000,))

In [0]:
x_train.shape, y_train.shape

((60000, 784), (60000, 10))